## Frankfurter
**Frankfurter** es una **API** de código abierto para tipos de cambio actuales e históricos publicados por el Banco Central Europeo (**European Central Bank**).


**_Documentación:_** https://www.frankfurter.app/docs/

### Endpoints

Para hacer llamadas a la **API** debemos usar la base del **endpoint**:

```html
https://api.frankfurter.app/

```

Y sobre esta base agregamos los parámetros para hacer la consulta:

```html
https://api.frankfurter.app/2000-01-01
```

Si la consulta tiene los parámetros correctos y tenemos los permisos para hacerla, tendremos una respuesta de 200, esto quiere decir que logramos comunicarnos con el **API** correctamente.


Con esta **API** podemos extraer los datos históricos de los tipos de cambio en Europa y también podremos hacer uso de un conversor de monedas integrado en la **API**.

In [ ]:
import numpy as np
import pandas as pd

import requests

from pprint import pprint

In [ ]:
# Versiones

print(f"numpy=={np.__version__}")
print(f"pandas=={pd.__version__}")
print(f"requests=={requests.__version__}")

In [ ]:
url = "https://api.frankfurter.app"

In [ ]:
response = requests.get(url)

In [ ]:
print(response)

In [ ]:
response.json()

Si no usamos ningún parámetro el **API** no nos retornará información. Veamos ahora como agregar parámetros y extraer información.

In [ ]:
url = "https://api.frankfurter.app"

date = "2000-01-01"

endpoint = f"{url}/{date}"

print(f"Endpoint: {endpoint}")

response = requests.get(endpoint)

response.json()

In [ ]:
# Con pprint
pprint(response.json())

In [ ]:
# Para agregar un rango de fechas podemos unir las dos fechas con ".."

url = "https://api.frankfurter.app"

date_1 = "2010-01-01"
date_2 = "2010-01-31"

endpoint = f"{url}/{date_1}..{date_2}"

print(f"Endpoint: {endpoint}")

response = requests.get(endpoint)

response.json()

In [ ]:
# Con pprint

pprint(response.json())

In [ ]:
# Para retornar los tipos de cambio del día de hoy tenemos un endpoint 

endpoint = f"{url}/latest"

print(f"Endpoint: {endpoint}")

response = requests.get(endpoint)

response.json()

In [ ]:
# Y si quisieramos retornar los tipos de cambio desde cierta fecha hasta el día de hoy

url = "https://api.frankfurter.app"

date_1 = "2010-01-01"

endpoint = f"{url}/{date_1}.."

print(f"Endpoint: {endpoint}")

response = requests.get(endpoint)

response.json()

In [ ]:
pprint(response.json())

En la documentación de **Frankfurter** dice:

**Frankfurter returns all data points for up to 90 days. Above that, it starts sampling by week or month based on the breadth of the date range.**

Lo que significa que si hacemos una consulta con muchas fechas el **API** automaticamente reducirá el número de fechas acorde al total para no retornar demasiadas.

In [ ]:
# Total de fechas de la consulta anterior

len(response.json()["rates"].keys())

# La consulta anterior tenia fecha de comienzo el 2010-01-01 y terminaba el día de hoy

In [ ]:
# Fechas de la consulta

sorted(response.json()["rates"].keys())

In [ ]:
# Usando la API también podemos comparar los euros con alguna otra moneda usando el parámetro "to"

url = "https://api.frankfurter.app"

to_conversion = "GBP"

endpoint = f"{url}/latest?to={to_conversion}"

print(f"Endpoint: {endpoint}")

response = requests.get(endpoint)

pprint(response.json())

In [ ]:
# Y también podemos combinar los parámetros

url = "https://api.frankfurter.app"

date_1 = "2010-01-01"
date_2 = "2010-01-31"

to_conversion = "GBP,USD"

endpoint = f"{url}/{date_1}..{date_2}?to={to_conversion}"

print(f"Endpoint: {endpoint}")

response = requests.get(endpoint)

pprint(response.json())

### Currencies

In [ ]:
# Endpoint especial para obtener las monedas que usan en Frankfurter y su significado

url = "https://api.frankfurter.app"

endpoint = f"{url}/currencies"

print(f"Endpoint: {endpoint}")

response = requests.get(endpoint)

pprint(response.json())

In [ ]:
df_currencies = pd.DataFrame(response.json().items(), columns = ["abr", "currency"])

df_currencies

### Conversion

In [ ]:
# Podemos usar Frankfurter como una calculadora de conversión de monedas
# Podemos hacer la conversión de una sola fecha

url = "https://api.frankfurter.app"

amount = 10

from_currency = "USD"

to_currency = "GBP"

endpoint = f"{url}/latest?amount={amount}&from={from_currency}&to={to_currency}"

print(f"Endpoint: {endpoint}")

response = requests.get(endpoint)

pprint(response.json())

In [ ]:
# O de muchas fechas

url = "https://api.frankfurter.app"

amount = 10

from_currency = "USD"

to_currency = "GBP"

date_1 = "2010-01-01"
date_2 = "2010-01-31"

endpoint = f"{url}/{date_1}..{date_2}?amount={amount}&from={from_currency}&to={to_currency}"

print(f"Endpoint: {endpoint}")

response = requests.get(endpoint)

pprint(response.json())

### Ejemplo: Evolución del EUR comparado con USD

Vamos a hacer una gráfica que muestre la evolución del euro comparado con el dolar este año.

In [ ]:
url = "https://api.frankfurter.app"

año = "2023"

date = f"{año}-01-01"

to_currency = "USD"

#### 1. Primero debemos extraer los datos desde el primer día de este año hasta el día de hoy.

In [ ]:
endpoint = f"{url}/{date}..?to={to_currency}"

print(f"Endpoint: {endpoint}")

response = requests.get(endpoint)

pprint(response.json())

data = response.json()

#### 2. Vamos a manipular los datos para tenerlos en una lista o array.

In [ ]:
from datetime import datetime

In [ ]:
data["rates"]

In [ ]:
fechas = list(data["rates"].keys())

fechas = [datetime.strptime(fecha, "%Y-%m-%d") for fecha in fechas]

fechas

In [ ]:
# [x[to_currency] for x in data["rates"].values()]

In [ ]:
dolares = [data["rates"][fecha][to_currency] for fecha in data["rates"].keys()]

dolares

In [ ]:
# dolares = list()

# for x in data["rates"].values():
    
#     dolares.append(x["USD"])
    
# dolares

#### 3. Visualización

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
max(dolares)

In [ ]:
np.argmax(dolares)

In [ ]:
fechas[np.argmax(dolares)]

In [ ]:
fechas[52]

In [ ]:
# Tamaño de la figura
plt.figure(figsize = (18, 10))

# Plot (función especial que trabaja con fechas)
plt.plot_date(x = fechas, y = dolares, fmt = "--")

# max
plt.plot(fechas[np.argmax(dolares)],
         np.max(dolares),
         color      = "green",
         marker     = "o",
         markersize = 10,
         label      = f"max: {np.max(dolares)}")

# min
plt.plot(fechas[np.argmin(dolares)],
         np.min(dolares),
         color      = "red",
         marker     = "o",
         markersize = 10,
         label      = f"min: {np.min(dolares)}")

# Evita que se sobrepongan las fechas
plt.gcf().autofmt_xdate()

# Etiquetas
plt.legend()
plt.xlabel("Fechas", fontsize = 20)
plt.ylabel("USD", fontsize = 20)
plt.title("Relación EUR - USD 2022", fontsize = 20)

plt.grid()
plt.show()

In [ ]:
print(f"Max: {max(dolares)}")
print(f"Min: {min(dolares)}")

#### Plotly

In [ ]:
column_names = ["date", "dolars"]
column_data = [fechas, dolares]

dict_df = {name : data for name, data in zip(column_names, column_data)}

# dict_df

In [ ]:
df_dolares = pd.DataFrame(dict_df)

df_dolares

In [ ]:
import plotly.express as px

px.line(data_frame = df_dolares,
        x          = "date",
        y          = "dolars",
        title      = "USD - EUR Relationship")

In [ ]:
################################################################################################################################